# FBT Entertainment Expense Classification Pipeline
## Rabobank Australia - FBT Year Ended 31 March 2025

### Pipeline Features:
- Comprehensive data cleaning and preprocessing
- Location extraction and geocoding
- Travel distance and time estimation
- Advanced text feature engineering
- Supervised classification using ME? Y/N labels
- Optional unsupervised clustering for pattern discovery
- Multiple model comparison with hyperparameter tuning

---

## 1. Configuration

In [14]:
# =============================================================================
# CONFIGURATION
# =============================================================================

CONFIG = {
    # File paths
    'raw_data_path': 'data/data_raw_2024-25.xlsx',
    'wp_files': [
        'data/WP_1_Apr_to_Dec_24_FBT_ent_acc.xlsx',
        'data/WP_2_Apr_to_Dec_24_FBT_ent_acc.xlsx',
        'data/WP_3_Jan_to_Mar_25_FBT_ent_acc.xlsx',
        'data/WP_4_Jan_to_Mar_25_FBT_ent_acc.xlsx'
    ],
    
    # Reference location for distance calculations (Sydney CBD - Rabobank office)
    'reference_location': {'lat': -33.8688, 'lon': 151.2093, 'name': 'Sydney CBD'},
    
    # Classification mode: 'supervised' or 'unsupervised'
    'mode': 'supervised',
    
    # Enable/disable features
    'enable_geocoding': True,
    'enable_clustering': True,
    
    # Clustering settings
    'n_clusters': 8,
    
    # Model training
    'test_size': 0.2,
    'random_state': 42,
    'cv_folds': 5,
    
    # Text vectorization
    'max_features': 5000,
    'ngram_range': (1, 3),
    'min_df': 2,
    'max_df': 0.95,
    
    # Output paths
    'model_output': 'fbt_classifier_pipeline.joblib',
    'encoder_output': 'fbt_label_encoder.joblib',
    'predictions_output': 'fbt_predictions.csv'
}

## 2. Imports and Setup

In [15]:
import pandas as pd
import numpy as np
import re
import warnings
from collections import Counter
from typing import List, Dict, Tuple, Optional
import math
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Text processing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Model selection
from sklearn.model_selection import (
    train_test_split, cross_val_score, StratifiedKFold,
    GridSearchCV, RandomizedSearchCV, learning_curve
)

# Supervised models
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, ExtraTreesClassifier, VotingClassifier,
    BaggingClassifier, StackingClassifier
)
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV

# Unsupervised models
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.decomposition import PCA, TruncatedSVD, LatentDirichletAllocation, NMF
from sklearn.manifold import TSNE

# Evaluation
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_recall_fscore_support, roc_auc_score, f1_score,
    ConfusionMatrixDisplay, roc_curve, precision_recall_curve,
    silhouette_score, calinski_harabasz_score, davies_bouldin_score
)

# Pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# Imbalanced learning
try:
    from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
    from imblearn.under_sampling import RandomUnderSampler, TomekLinks
    from imblearn.combine import SMOTETomek
    from imblearn.pipeline import Pipeline as ImbPipeline
    IMBLEARN_AVAILABLE = True
except ImportError:
    IMBLEARN_AVAILABLE = False
    print("Note: imbalanced-learn not installed. Run: pip install imbalanced-learn")

# Geocoding
try:
    from geopy.geocoders import Nominatim
    from geopy.distance import geodesic
    from geopy.exc import GeocoderTimedOut
    GEOPY_AVAILABLE = True
except ImportError:
    GEOPY_AVAILABLE = False
    print("Note: geopy not installed. Run: pip install geopy")

# Model persistence
import joblib

print("All imports successful")
print(f"imbalanced-learn available: {IMBLEARN_AVAILABLE}")
print(f"geopy available: {GEOPY_AVAILABLE}")

All imports successful
imbalanced-learn available: True
geopy available: True


## 3. Australian Location Database

In [16]:
# =============================================================================
# AUSTRALIAN LOCATIONS DATABASE
# Pre-defined coordinates to avoid excessive API calls
# =============================================================================

AUSTRALIAN_LOCATIONS = {
    # Major Cities
    'sydney': {'lat': -33.8688, 'lon': 151.2093, 'state': 'NSW', 'type': 'city'},
    'melbourne': {'lat': -37.8136, 'lon': 144.9631, 'state': 'VIC', 'type': 'city'},
    'brisbane': {'lat': -27.4698, 'lon': 153.0251, 'state': 'QLD', 'type': 'city'},
    'perth': {'lat': -31.9505, 'lon': 115.8605, 'state': 'WA', 'type': 'city'},
    'adelaide': {'lat': -34.9285, 'lon': 138.6007, 'state': 'SA', 'type': 'city'},
    'hobart': {'lat': -42.8821, 'lon': 147.3272, 'state': 'TAS', 'type': 'city'},
    'darwin': {'lat': -12.4634, 'lon': 130.8456, 'state': 'NT', 'type': 'city'},
    'canberra': {'lat': -35.2809, 'lon': 149.1300, 'state': 'ACT', 'type': 'city'},
    
    # Regional NSW
    'newcastle': {'lat': -32.9283, 'lon': 151.7817, 'state': 'NSW', 'type': 'regional'},
    'wollongong': {'lat': -34.4278, 'lon': 150.8931, 'state': 'NSW', 'type': 'regional'},
    'dubbo': {'lat': -32.2569, 'lon': 148.6011, 'state': 'NSW', 'type': 'regional'},
    'tamworth': {'lat': -31.0830, 'lon': 150.9170, 'state': 'NSW', 'type': 'regional'},
    'wagga wagga': {'lat': -35.1082, 'lon': 147.3598, 'state': 'NSW', 'type': 'regional'},
    'wagga': {'lat': -35.1082, 'lon': 147.3598, 'state': 'NSW', 'type': 'regional'},
    'orange': {'lat': -33.2840, 'lon': 149.1004, 'state': 'NSW', 'type': 'regional'},
    'bathurst': {'lat': -33.4190, 'lon': 149.5778, 'state': 'NSW', 'type': 'regional'},
    'albury': {'lat': -36.0737, 'lon': 146.9135, 'state': 'NSW', 'type': 'regional'},
    'broken hill': {'lat': -31.9539, 'lon': 141.4428, 'state': 'NSW', 'type': 'regional'},
    'armidale': {'lat': -30.5150, 'lon': 151.6500, 'state': 'NSW', 'type': 'regional'},
    'lismore': {'lat': -28.8120, 'lon': 153.2790, 'state': 'NSW', 'type': 'regional'},
    'grafton': {'lat': -29.6850, 'lon': 152.9330, 'state': 'NSW', 'type': 'regional'},
    'coffs harbour': {'lat': -30.2963, 'lon': 153.1135, 'state': 'NSW', 'type': 'regional'},
    'port macquarie': {'lat': -31.4333, 'lon': 152.9000, 'state': 'NSW', 'type': 'regional'},
    'moree': {'lat': -29.4640, 'lon': 149.8470, 'state': 'NSW', 'type': 'regional'},
    'goulburn': {'lat': -34.7547, 'lon': 149.7186, 'state': 'NSW', 'type': 'regional'},
    'nowra': {'lat': -34.8800, 'lon': 150.6000, 'state': 'NSW', 'type': 'regional'},
    'griffith': {'lat': -34.2890, 'lon': 146.0400, 'state': 'NSW', 'type': 'regional'},
    'parkes': {'lat': -33.1370, 'lon': 148.1750, 'state': 'NSW', 'type': 'regional'},
    'forbes': {'lat': -33.3850, 'lon': 148.0170, 'state': 'NSW', 'type': 'regional'},
    'cowra': {'lat': -33.8330, 'lon': 148.6830, 'state': 'NSW', 'type': 'regional'},
    'young': {'lat': -34.3130, 'lon': 148.3000, 'state': 'NSW', 'type': 'regional'},
    'mudgee': {'lat': -32.6000, 'lon': 149.5830, 'state': 'NSW', 'type': 'regional'},
    'cobar': {'lat': -31.4950, 'lon': 145.8380, 'state': 'NSW', 'type': 'regional'},
    'bourke': {'lat': -30.0900, 'lon': 145.9380, 'state': 'NSW', 'type': 'regional'},
    'walgett': {'lat': -30.0200, 'lon': 148.1170, 'state': 'NSW', 'type': 'regional'},
    'narrabri': {'lat': -30.3250, 'lon': 149.7830, 'state': 'NSW', 'type': 'regional'},
    'gunnedah': {'lat': -30.9800, 'lon': 150.2500, 'state': 'NSW', 'type': 'regional'},
    'inverell': {'lat': -29.7750, 'lon': 151.1170, 'state': 'NSW', 'type': 'regional'},
    'glen innes': {'lat': -29.7330, 'lon': 151.7330, 'state': 'NSW', 'type': 'regional'},
    'tenterfield': {'lat': -29.0500, 'lon': 152.0170, 'state': 'NSW', 'type': 'regional'},
    
    # Regional QLD
    'gold coast': {'lat': -28.0167, 'lon': 153.4000, 'state': 'QLD', 'type': 'regional'},
    'sunshine coast': {'lat': -26.6500, 'lon': 153.0667, 'state': 'QLD', 'type': 'regional'},
    'cairns': {'lat': -16.9186, 'lon': 145.7781, 'state': 'QLD', 'type': 'regional'},
    'townsville': {'lat': -19.2590, 'lon': 146.8169, 'state': 'QLD', 'type': 'regional'},
    'toowoomba': {'lat': -27.5598, 'lon': 151.9507, 'state': 'QLD', 'type': 'regional'},
    'rockhampton': {'lat': -23.3791, 'lon': 150.5100, 'state': 'QLD', 'type': 'regional'},
    'mackay': {'lat': -21.1411, 'lon': 149.1861, 'state': 'QLD', 'type': 'regional'},
    'bundaberg': {'lat': -24.8661, 'lon': 152.3489, 'state': 'QLD', 'type': 'regional'},
    'gladstone': {'lat': -23.8427, 'lon': 151.2555, 'state': 'QLD', 'type': 'regional'},
    'hervey bay': {'lat': -25.2900, 'lon': 152.8500, 'state': 'QLD', 'type': 'regional'},
    'roma': {'lat': -26.5700, 'lon': 148.7850, 'state': 'QLD', 'type': 'regional'},
    'charleville': {'lat': -26.4030, 'lon': 146.2430, 'state': 'QLD', 'type': 'regional'},
    'longreach': {'lat': -23.4420, 'lon': 144.2500, 'state': 'QLD', 'type': 'regional'},
    'mount isa': {'lat': -20.7256, 'lon': 139.4927, 'state': 'QLD', 'type': 'regional'},
    'cloncurry': {'lat': -20.7050, 'lon': 140.5060, 'state': 'QLD', 'type': 'regional'},
    'emerald': {'lat': -23.5270, 'lon': 148.1640, 'state': 'QLD', 'type': 'regional'},
    'dalby': {'lat': -27.1810, 'lon': 151.2650, 'state': 'QLD', 'type': 'regional'},
    'kingaroy': {'lat': -26.5400, 'lon': 151.8400, 'state': 'QLD', 'type': 'regional'},
    'warwick': {'lat': -28.2150, 'lon': 152.0350, 'state': 'QLD', 'type': 'regional'},
    'stanthorpe': {'lat': -28.6570, 'lon': 151.9350, 'state': 'QLD', 'type': 'regional'},
    'goondiwindi': {'lat': -28.5470, 'lon': 150.3100, 'state': 'QLD', 'type': 'regional'},
    'st george': {'lat': -28.0370, 'lon': 148.5830, 'state': 'QLD', 'type': 'regional'},
    'cunnamulla': {'lat': -28.0670, 'lon': 145.6830, 'state': 'QLD', 'type': 'regional'},
    'charters towers': {'lat': -20.0760, 'lon': 146.2610, 'state': 'QLD', 'type': 'regional'},
    'ayr': {'lat': -19.5740, 'lon': 147.4040, 'state': 'QLD', 'type': 'regional'},
    'ingham': {'lat': -18.6500, 'lon': 146.1670, 'state': 'QLD', 'type': 'regional'},
    'innisfail': {'lat': -17.5240, 'lon': 146.0330, 'state': 'QLD', 'type': 'regional'},
    'atherton': {'lat': -17.2670, 'lon': 145.4830, 'state': 'QLD', 'type': 'regional'},
    'mareeba': {'lat': -17.0000, 'lon': 145.4330, 'state': 'QLD', 'type': 'regional'},
    'biloela': {'lat': -24.4000, 'lon': 150.5170, 'state': 'QLD', 'type': 'regional'},
    'monto': {'lat': -24.8670, 'lon': 151.1170, 'state': 'QLD', 'type': 'regional'},
    'gayndah': {'lat': -25.6300, 'lon': 151.6170, 'state': 'QLD', 'type': 'regional'},
    'murgon': {'lat': -26.2400, 'lon': 151.9400, 'state': 'QLD', 'type': 'regional'},
    
    # Regional VIC
    'geelong': {'lat': -38.1499, 'lon': 144.3617, 'state': 'VIC', 'type': 'regional'},
    'ballarat': {'lat': -37.5622, 'lon': 143.8503, 'state': 'VIC', 'type': 'regional'},
    'bendigo': {'lat': -36.7570, 'lon': 144.2794, 'state': 'VIC', 'type': 'regional'},
    'shepparton': {'lat': -36.3833, 'lon': 145.4000, 'state': 'VIC', 'type': 'regional'},
    'mildura': {'lat': -34.2087, 'lon': 142.1311, 'state': 'VIC', 'type': 'regional'},
    'warrnambool': {'lat': -38.3818, 'lon': 142.4876, 'state': 'VIC', 'type': 'regional'},
    'wodonga': {'lat': -36.1217, 'lon': 146.8883, 'state': 'VIC', 'type': 'regional'},
    'horsham': {'lat': -36.7117, 'lon': 142.2000, 'state': 'VIC', 'type': 'regional'},
    'wangaratta': {'lat': -36.3578, 'lon': 146.3122, 'state': 'VIC', 'type': 'regional'},
    'sale': {'lat': -38.1000, 'lon': 147.0667, 'state': 'VIC', 'type': 'regional'},
    'traralgon': {'lat': -38.1950, 'lon': 146.5400, 'state': 'VIC', 'type': 'regional'},
    'bairnsdale': {'lat': -37.8230, 'lon': 147.6100, 'state': 'VIC', 'type': 'regional'},
    'echuca': {'lat': -36.1300, 'lon': 144.7500, 'state': 'VIC', 'type': 'regional'},
    'swan hill': {'lat': -35.3380, 'lon': 143.5500, 'state': 'VIC', 'type': 'regional'},
    'hamilton': {'lat': -37.7440, 'lon': 142.0220, 'state': 'VIC', 'type': 'regional'},
    'colac': {'lat': -38.3400, 'lon': 143.5900, 'state': 'VIC', 'type': 'regional'},
    'ararat': {'lat': -37.2840, 'lon': 142.9300, 'state': 'VIC', 'type': 'regional'},
    'stawell': {'lat': -37.0560, 'lon': 142.7800, 'state': 'VIC', 'type': 'regional'},
    'castlemaine': {'lat': -37.0670, 'lon': 144.2170, 'state': 'VIC', 'type': 'regional'},
    'kyneton': {'lat': -37.2500, 'lon': 144.4500, 'state': 'VIC', 'type': 'regional'},
    'seymour': {'lat': -37.0260, 'lon': 145.1390, 'state': 'VIC', 'type': 'regional'},
    'benalla': {'lat': -36.5520, 'lon': 145.9830, 'state': 'VIC', 'type': 'regional'},
    'cobram': {'lat': -35.9200, 'lon': 145.6500, 'state': 'VIC', 'type': 'regional'},
    'yarrawonga': {'lat': -36.0300, 'lon': 146.0000, 'state': 'VIC', 'type': 'regional'},
    'kyabram': {'lat': -36.3170, 'lon': 145.0500, 'state': 'VIC', 'type': 'regional'},
    'rochester': {'lat': -36.3670, 'lon': 144.7000, 'state': 'VIC', 'type': 'regional'},
    'kerang': {'lat': -35.7330, 'lon': 143.9170, 'state': 'VIC', 'type': 'regional'},
    
    # Regional SA
    'mount gambier': {'lat': -37.8297, 'lon': 140.7811, 'state': 'SA', 'type': 'regional'},
    'whyalla': {'lat': -33.0333, 'lon': 137.5167, 'state': 'SA', 'type': 'regional'},
    'port lincoln': {'lat': -34.7333, 'lon': 135.8500, 'state': 'SA', 'type': 'regional'},
    'port augusta': {'lat': -32.4936, 'lon': 137.7825, 'state': 'SA', 'type': 'regional'},
    'port pirie': {'lat': -33.1858, 'lon': 138.0169, 'state': 'SA', 'type': 'regional'},
    'murray bridge': {'lat': -35.1197, 'lon': 139.2756, 'state': 'SA', 'type': 'regional'},
    'renmark': {'lat': -34.1760, 'lon': 140.7470, 'state': 'SA', 'type': 'regional'},
    'berri': {'lat': -34.2830, 'lon': 140.6000, 'state': 'SA', 'type': 'regional'},
    'loxton': {'lat': -34.4500, 'lon': 140.5670, 'state': 'SA', 'type': 'regional'},
    'kadina': {'lat': -33.9670, 'lon': 137.7170, 'state': 'SA', 'type': 'regional'},
    'clare': {'lat': -33.8330, 'lon': 138.6000, 'state': 'SA', 'type': 'regional'},
    'tanunda': {'lat': -34.5230, 'lon': 138.9600, 'state': 'SA', 'type': 'regional'},
    'nuriootpa': {'lat': -34.4670, 'lon': 139.0000, 'state': 'SA', 'type': 'regional'},
    'naracoorte': {'lat': -36.9500, 'lon': 140.7330, 'state': 'SA', 'type': 'regional'},
    'bordertown': {'lat': -36.3100, 'lon': 140.7700, 'state': 'SA', 'type': 'regional'},
    
    # Regional WA
    'bunbury': {'lat': -33.3261, 'lon': 115.6394, 'state': 'WA', 'type': 'regional'},
    'geraldton': {'lat': -28.7775, 'lon': 114.6147, 'state': 'WA', 'type': 'regional'},
    'kalgoorlie': {'lat': -30.7489, 'lon': 121.4658, 'state': 'WA', 'type': 'regional'},
    'albany': {'lat': -35.0275, 'lon': 117.8839, 'state': 'WA', 'type': 'regional'},
    'mandurah': {'lat': -32.5269, 'lon': 115.7217, 'state': 'WA', 'type': 'regional'},
    'broome': {'lat': -17.9614, 'lon': 122.2359, 'state': 'WA', 'type': 'regional'},
    'karratha': {'lat': -20.7361, 'lon': 116.8467, 'state': 'WA', 'type': 'regional'},
    'port hedland': {'lat': -20.3108, 'lon': 118.5753, 'state': 'WA', 'type': 'regional'},
    'esperance': {'lat': -33.8611, 'lon': 121.8919, 'state': 'WA', 'type': 'regional'},
    'carnarvon': {'lat': -24.8840, 'lon': 113.6590, 'state': 'WA', 'type': 'regional'},
    'kununurra': {'lat': -15.7730, 'lon': 128.7380, 'state': 'WA', 'type': 'regional'},
    'collie': {'lat': -33.3600, 'lon': 116.1500, 'state': 'WA', 'type': 'regional'},
    'katanning': {'lat': -33.6900, 'lon': 117.5500, 'state': 'WA', 'type': 'regional'},
    'narrogin': {'lat': -32.9330, 'lon': 117.1830, 'state': 'WA', 'type': 'regional'},
    'merredin': {'lat': -31.4830, 'lon': 118.2830, 'state': 'WA', 'type': 'regional'},
    'northam': {'lat': -31.6500, 'lon': 116.6670, 'state': 'WA', 'type': 'regional'},
    'moora': {'lat': -30.6400, 'lon': 116.0170, 'state': 'WA', 'type': 'regional'},
    'dalwallinu': {'lat': -30.2770, 'lon': 116.6630, 'state': 'WA', 'type': 'regional'},
    
    # Regional NT
    'alice springs': {'lat': -23.6980, 'lon': 133.8807, 'state': 'NT', 'type': 'regional'},
    'katherine': {'lat': -14.4650, 'lon': 132.2635, 'state': 'NT', 'type': 'regional'},
    'tennant creek': {'lat': -19.6497, 'lon': 134.1917, 'state': 'NT', 'type': 'regional'},
    
    # Regional TAS
    'launceston': {'lat': -41.4332, 'lon': 147.1441, 'state': 'TAS', 'type': 'regional'},
    'devonport': {'lat': -41.1760, 'lon': 146.3510, 'state': 'TAS', 'type': 'regional'},
    'burnie': {'lat': -41.0556, 'lon': 145.9056, 'state': 'TAS', 'type': 'regional'},
    
    # Airports (IATA codes)
    'syd': {'lat': -33.9399, 'lon': 151.1753, 'state': 'NSW', 'type': 'airport'},
    'mel': {'lat': -37.6690, 'lon': 144.8410, 'state': 'VIC', 'type': 'airport'},
    'bne': {'lat': -27.3942, 'lon': 153.1218, 'state': 'QLD', 'type': 'airport'},
    'per': {'lat': -31.9403, 'lon': 115.9670, 'state': 'WA', 'type': 'airport'},
    'adl': {'lat': -34.9450, 'lon': 138.5306, 'state': 'SA', 'type': 'airport'},
    'cbr': {'lat': -35.3069, 'lon': 149.1950, 'state': 'ACT', 'type': 'airport'},
    'ool': {'lat': -28.1644, 'lon': 153.5047, 'state': 'QLD', 'type': 'airport'},  # Gold Coast
    'cns': {'lat': -16.8858, 'lon': 145.7553, 'state': 'QLD', 'type': 'airport'},  # Cairns
    'tsv': {'lat': -19.2525, 'lon': 146.7656, 'state': 'QLD', 'type': 'airport'},  # Townsville
    'dbo': {'lat': -32.2167, 'lon': 148.5747, 'state': 'NSW', 'type': 'airport'},  # Dubbo
    'ntl': {'lat': -32.7950, 'lon': 151.8342, 'state': 'NSW', 'type': 'airport'},  # Newcastle
    'rce': {'lat': -23.3819, 'lon': 150.4753, 'state': 'QLD', 'type': 'airport'},  # Rockhampton
    'mky': {'lat': -21.1717, 'lon': 149.1797, 'state': 'QLD', 'type': 'airport'},  # Mackay
    'isa': {'lat': -20.6639, 'lon': 139.4886, 'state': 'QLD', 'type': 'airport'},  # Mount Isa
    'eme': {'lat': -23.5675, 'lon': 148.1792, 'state': 'QLD', 'type': 'airport'},  # Emerald
    'lre': {'lat': -23.4342, 'lon': 144.2797, 'state': 'QLD', 'type': 'airport'},  # Longreach
    
    # International destinations (common business travel)
    'utrecht': {'lat': 52.0907, 'lon': 5.1214, 'state': 'NL', 'type': 'international'},
    'amsterdam': {'lat': 52.3676, 'lon': 4.9041, 'state': 'NL', 'type': 'international'},
    'netherlands': {'lat': 52.1326, 'lon': 5.2913, 'state': 'NL', 'type': 'international'},
    'singapore': {'lat': 1.3521, 'lon': 103.8198, 'state': 'SG', 'type': 'international'},
    'hong kong': {'lat': 22.3193, 'lon': 114.1694, 'state': 'HK', 'type': 'international'},
    'new zealand': {'lat': -40.9006, 'lon': 174.8860, 'state': 'NZ', 'type': 'international'},
    'auckland': {'lat': -36.8509, 'lon': 174.7645, 'state': 'NZ', 'type': 'international'},
    'wellington': {'lat': -41.2924, 'lon': 174.7787, 'state': 'NZ', 'type': 'international'},
    'london': {'lat': 51.5074, 'lon': -0.1278, 'state': 'UK', 'type': 'international'},
    'boston': {'lat': 42.3601, 'lon': -71.0589, 'state': 'US', 'type': 'international'},
    'new york': {'lat': 40.7128, 'lon': -74.0060, 'state': 'US', 'type': 'international'},
    'tokyo': {'lat': 35.6762, 'lon': 139.6503, 'state': 'JP', 'type': 'international'},
    'beijing': {'lat': 39.9042, 'lon': 116.4074, 'state': 'CN', 'type': 'international'},
    'shanghai': {'lat': 31.2304, 'lon': 121.4737, 'state': 'CN', 'type': 'international'},
    'jakarta': {'lat': -6.2088, 'lon': 106.8456, 'state': 'ID', 'type': 'international'},
    'manila': {'lat': 14.5995, 'lon': 120.9842, 'state': 'PH', 'type': 'international'},
    'bangkok': {'lat': 13.7563, 'lon': 100.5018, 'state': 'TH', 'type': 'international'},
    'mumbai': {'lat': 19.0760, 'lon': 72.8777, 'state': 'IN', 'type': 'international'},
    'dubai': {'lat': 25.2048, 'lon': 55.2708, 'state': 'AE', 'type': 'international'},
}

# State abbreviations
STATE_MAPPING = {
    'nsw': 'NSW', 'new south wales': 'NSW',
    'vic': 'VIC', 'victoria': 'VIC',
    'qld': 'QLD', 'queensland': 'QLD',
    'wa': 'WA', 'western australia': 'WA',
    'sa': 'SA', 'south australia': 'SA',
    'tas': 'TAS', 'tasmania': 'TAS',
    'nt': 'NT', 'northern territory': 'NT',
    'act': 'ACT', 'australian capital territory': 'ACT'
}

print(f"Loaded {len(AUSTRALIAN_LOCATIONS)} locations in database")

Loaded 174 locations in database


## 4. Location Extraction and Geocoding

In [17]:
class LocationExtractor:
    """
    Extract and geocode locations from expense descriptions.
    """
    
    def __init__(self, reference_location: Dict = None):
        self.locations_db = AUSTRALIAN_LOCATIONS
        self.reference = reference_location or CONFIG['reference_location']
        self.cache = {}
        
        # Build pattern for location matching
        location_names = sorted(self.locations_db.keys(), key=len, reverse=True)
        escaped_names = [re.escape(name) for name in location_names]
        self.location_pattern = re.compile(
            r'\b(' + '|'.join(escaped_names) + r')\b',
            re.IGNORECASE
        )
        
        # Initialize geocoder for unknown locations (if available)
        if GEOPY_AVAILABLE:
            self.geocoder = Nominatim(user_agent="fbt_classifier")
        else:
            self.geocoder = None
    
    def extract_locations(self, text: str) -> List[str]:
        """
        Extract location names from text.
        """
        if pd.isna(text) or not text:
            return []
        
        text = str(text).lower()
        matches = self.location_pattern.findall(text)
        return list(set(matches))
    
    def get_coordinates(self, location_name: str) -> Optional[Dict]:
        """
        Get coordinates for a location name.
        """
        location_name = location_name.lower().strip()
        
        # Check cache first
        if location_name in self.cache:
            return self.cache[location_name]
        
        # Check database
        if location_name in self.locations_db:
            coords = self.locations_db[location_name]
            self.cache[location_name] = coords
            return coords
        
        # Try geocoding unknown locations (with rate limiting)
        if self.geocoder and CONFIG.get('enable_geocoding', False):
            try:
                result = self.geocoder.geocode(f"{location_name}, Australia", timeout=5)
                if result:
                    coords = {'lat': result.latitude, 'lon': result.longitude, 
                              'state': 'UNKNOWN', 'type': 'geocoded'}
                    self.cache[location_name] = coords
                    return coords
            except Exception:
                pass
        
        return None
    
    def calculate_distance(self, lat1: float, lon1: float, 
                           lat2: float, lon2: float) -> float:
        """
        Calculate distance between two points using Haversine formula.
        Returns distance in kilometers.
        """
        R = 6371  # Earth's radius in km
        
        lat1_rad = math.radians(lat1)
        lat2_rad = math.radians(lat2)
        delta_lat = math.radians(lat2 - lat1)
        delta_lon = math.radians(lon2 - lon1)
        
        a = (math.sin(delta_lat/2)**2 + 
             math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon/2)**2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        
        return R * c
    
    def estimate_travel_time(self, distance_km: float, travel_type: str = 'auto') -> float:
        """
        Estimate travel time in hours based on distance.
        """
        if distance_km <= 0:
            return 0
        
        # Rough estimates
        if travel_type == 'flight' or distance_km > 500:
            # Flight time estimate (800 km/h average + 2h airport time)
            return (distance_km / 800) + 2
        elif travel_type == 'car' or distance_km <= 500:
            # Driving estimate (80 km/h average including stops)
            return distance_km / 80
        else:
            # Default to car
            return distance_km / 80
    
    def extract_location_features(self, text: str) -> Dict:
        """
        Extract all location-related features from text.
        """
        features = {
            'locations_found': 0,
            'location_names': '',
            'primary_location': '',
            'primary_state': '',
            'primary_lat': np.nan,
            'primary_lon': np.nan,
            'distance_from_ref_km': np.nan,
            'estimated_travel_hours': np.nan,
            'is_international': 0,
            'is_regional': 0,
            'is_major_city': 0,
            'is_local': 0,  # Within 100km
            'is_interstate': 0,
            'is_remote': 0,  # > 500km
            'travel_category': 'unknown'
        }
        
        locations = self.extract_locations(text)
        features['locations_found'] = len(locations)
        
        if not locations:
            return features
        
        features['location_names'] = ', '.join(locations)
        
        # Use first found location as primary
        primary = locations[0]
        features['primary_location'] = primary
        
        coords = self.get_coordinates(primary)
        if coords:
            features['primary_lat'] = coords['lat']
            features['primary_lon'] = coords['lon']
            features['primary_state'] = coords.get('state', '')
            
            # Location type flags
            loc_type = coords.get('type', '')
            features['is_international'] = int(loc_type == 'international')
            features['is_regional'] = int(loc_type == 'regional')
            features['is_major_city'] = int(loc_type == 'city')
            
            # Calculate distance from reference
            distance = self.calculate_distance(
                self.reference['lat'], self.reference['lon'],
                coords['lat'], coords['lon']
            )
            features['distance_from_ref_km'] = round(distance, 2)
            
            # Travel time estimate
            travel_type = 'flight' if distance > 500 or features['is_international'] else 'car'
            features['estimated_travel_hours'] = round(
                self.estimate_travel_time(distance, travel_type), 2
            )
            
            # Distance categories
            features['is_local'] = int(distance <= 100)
            features['is_remote'] = int(distance > 500)
            features['is_interstate'] = int(
                coords.get('state', 'NSW') != 'NSW' and not features['is_international']
            )
            
            # Travel category
            if features['is_international']:
                features['travel_category'] = 'international'
            elif features['is_remote']:
                features['travel_category'] = 'remote'
            elif features['is_interstate']:
                features['travel_category'] = 'interstate'
            elif features['is_regional']:
                features['travel_category'] = 'regional'
            elif features['is_local']:
                features['travel_category'] = 'local'
            else:
                features['travel_category'] = 'domestic'
        
        return features

In [18]:
# Test location extraction
loc_extractor = LocationExtractor()

test_texts = [
    "Flight to Brisbane for client meeting",
    "Dinner at Roma with clients - Board Members Visit",
    "Trip to Utrecht for Global General Counsel Offsite",
    "Taxi to airport for Sydney trip",
    "Team lunch at local restaurant",
    "Travel to Dubbo for farm visit",
    "Client dinner in Melbourne CBD"
]

print("Location Extraction Test:")
print("=" * 70)
for text in test_texts:
    features = loc_extractor.extract_location_features(text)
    print(f"\nText: {text}")
    print(f"  Location: {features['primary_location']}")
    print(f"  Distance: {features['distance_from_ref_km']} km")
    print(f"  Travel Time: {features['estimated_travel_hours']} hrs")
    print(f"  Category: {features['travel_category']}")

Location Extraction Test:

Text: Flight to Brisbane for client meeting
  Location: brisbane
  Distance: 732.38 km
  Travel Time: 2.92 hrs
  Category: remote

Text: Dinner at Roma with clients - Board Members Visit
  Location: roma
  Distance: 844.27 km
  Travel Time: 3.06 hrs
  Category: remote

Text: Trip to Utrecht for Global General Counsel Offsite
  Location: utrecht
  Distance: 16641.8 km
  Travel Time: 22.8 hrs
  Category: international

Text: Taxi to airport for Sydney trip
  Location: sydney
  Distance: 0.0 km
  Travel Time: 0 hrs
  Category: local

Text: Team lunch at local restaurant
  Location: 
  Distance: nan km
  Travel Time: nan hrs
  Category: unknown

Text: Travel to Dubbo for farm visit
  Location: dubbo
  Distance: 301.98 km
  Travel Time: 3.77 hrs
  Category: regional

Text: Client dinner in Melbourne CBD
  Location: melbourne
  Distance: 713.43 km
  Travel Time: 2.89 hrs
  Category: remote


## 5. Data Loading

In [19]:
def load_raw_data(filepath: str) -> pd.DataFrame:
    """
    Load raw transaction data from all sheets.
    """
    print(f"Loading raw data from: {filepath}")
    xl = pd.ExcelFile(filepath)
    print(f"Found {len(xl.sheet_names)} sheets")
    
    dfs = []
    for sheet in xl.sheet_names:
        df = pd.read_excel(filepath, sheet_name=sheet)
        df['_source_sheet'] = sheet
        dfs.append(df)
        print(f"  - {sheet}: {len(df)} rows")
    
    combined = pd.concat(dfs, ignore_index=True)
    print(f"\nTotal: {len(combined)} rows")
    return combined


def parse_workpaper(filepath: str, sheet_name: str) -> Optional[pd.DataFrame]:
    """
    Parse a workpaper sheet to extract labeled transactions.
    """
    try:
        df = pd.read_excel(filepath, sheet_name=sheet_name, header=None)
        
        # Find header row
        header_idx = None
        for i, row in df.iterrows():
            row_str = ' '.join([str(v).upper() for v in row.values if pd.notna(v)])
            if 'BUSINESS_UNIT_CODE' in row_str:
                header_idx = i
                break
        
        if header_idx is None:
            return None
        
        # Set headers
        headers = df.iloc[header_idx].tolist()
        headers = [str(h).strip() if pd.notna(h) else f'_col_{i}' for i, h in enumerate(headers)]
        
        data = df.iloc[header_idx + 1:].copy()
        data.columns = headers
        
        # Find label columns
        col_mapping = {}
        for col in headers:
            col_lower = str(col).lower()
            if 'tax' in col_lower and 'desc' in col_lower:
                col_mapping[col] = 'tax_description'
            elif 'me?' in col_lower:
                col_mapping[col] = 'me_label'
        
        data = data.rename(columns=col_mapping)
        data['_source_file'] = filepath.split('/')[-1]
        data['_source_sheet'] = sheet_name
        
        return data
    except Exception as e:
        print(f"  Error: {e}")
        return None


def load_labeled_data(wp_files: List[str]) -> pd.DataFrame:
    """
    Load labeled data from all workpapers.
    """
    all_dfs = []
    
    for filepath in wp_files:
        print(f"\nProcessing: {filepath}")
        try:
            xl = pd.ExcelFile(filepath)
            for sheet in xl.sheet_names:
                if any(skip in sheet.lower() for skip in ['summary', 'trial balance']):
                    continue
                
                df = parse_workpaper(filepath, sheet)
                if df is not None and len(df) > 0:
                    has_me = 'me_label' in df.columns
                    print(f"  - {sheet}: {len(df)} rows (has labels: {has_me})")
                    all_dfs.append(df)
        except Exception as e:
            print(f"  Error: {e}")
    
    if not all_dfs:
        return pd.DataFrame()
    
    combined = pd.concat(all_dfs, ignore_index=True)
    print(f"\nTotal labeled: {len(combined)} rows")
    return combined

In [20]:
# Load data
raw_data = load_raw_data(CONFIG['raw_data_path'])
labeled_data = load_labeled_data(CONFIG['wp_files'])

Loading raw data from: data/data_raw_2024-25.xlsx
Found 14 sheets
  - 5130006000: 224 rows
  - 5130002000: 4347 rows
  - 5130005000: 3673 rows
  - 5130004000: 27108 rows
  - 5130001000: 3097 rows
  - 5130003000: 1165 rows
  - 5140001010: 349 rows
  - 5140001030: 114 rows
  - 5160001000: 570 rows
  - 5160007000: 76 rows
  - 5160008000: 48 rows
  - 5160010000: 247 rows
  - 5160011000: 1311 rows
  - Misc: 5062 rows

Total: 47391 rows

Processing: data/WP_1_Apr_to_Dec_24_FBT_ent_acc.xlsx
  - Travel&lodging: 33218 rows (has labels: True)
  - Meals Exp: 22080 rows (has labels: True)
  - Lodging Exp: 3083 rows (has labels: True)
  - Dom Pub Trnsprt: 3699 rows (has labels: True)
  - Travel Exp Abroad: 1013 rows (has labels: True)
  - Ent Exp: 163 rows (has labels: True)

Processing: data/WP_2_Apr_to_Dec_24_FBT_ent_acc.xlsx
  - Ext Training: 296 rows (has labels: True)
  - Training Travel&lodging: 101 rows (has labels: True)
  - Publicity&Advertising: 401 rows (has labels: True)
  - Sponsoring:

## 6. Data Cleaning

In [23]:
class DataCleaner:
    """
    Comprehensive data cleaning.
    """
    
    def __init__(self):
        self.stats = {}
    
    def standardize_columns(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        df = df.loc[:, ~df.columns.str.contains('^Unnamed|^_col_', case=False, na=False)]
        
        new_cols = []
        for col in df.columns:
            new_col = str(col).strip()
            new_col = re.sub(r'\s+', '_', new_col)
            new_col = re.sub(r'[^a-zA-Z0-9_]', '', new_col)
            new_col = new_col.upper()
            new_cols.append(new_col)
        
        df.columns = new_cols
        return df
    
    def clean_text(self, series: pd.Series) -> pd.Series:
        series = series.fillna('').astype(str).str.strip()
        series = series.str.replace(r'\s+', ' ', regex=True)
        series = series.replace(['nan', 'NaN', 'None', ''], np.nan)
        return series
    
    def clean_me_label(self, series: pd.Series) -> pd.Series:
        series = series.fillna('').astype(str).str.strip().str.upper()
        mapping = {
            'Y': 'Y', 'YES': 'Y', '1': 'Y', 'TRUE': 'Y',
            'N': 'N', 'NO': 'N', '0': 'N', 'FALSE': 'N'
        }
        return series.map(lambda x: mapping.get(x, np.nan))
    
    def clean(self, df: pd.DataFrame) -> pd.DataFrame:
        print(f"Cleaning {len(df)} rows...")
        initial = len(df)
        
        df = self.standardize_columns(df)
        
        # Clean text columns
        text_cols = ['PURPOSE', 'CHARGE_DESCRIPTION', 'LINE_DESCR', 
                     'DESCRIPTION', 'INVOICE_DESCR', 'VENDOR_NAME', 'DESCR']
        for col in text_cols:
            if col in df.columns:
                df[col] = self.clean_text(df[col])
        
        # Clean numeric
        if 'BASE_AMOUNT' in df.columns:
            df['BASE_AMOUNT'] = pd.to_numeric(df['BASE_AMOUNT'], errors='coerce')
        
        # Clean dates
        if 'JOURNAL_DATE' in df.columns:
            df['JOURNAL_DATE'] = pd.to_datetime(df['JOURNAL_DATE'], errors='coerce')
        
        # Clean labels
        if 'ME_LABEL' in df.columns:
            df['ME_LABEL'] = self.clean_me_label(df['ME_LABEL'])
        if 'me_label' in df.columns:
            df['me_label'] = self.clean_me_label(df['me_label'])
        
        # Remove duplicates
        dedup_cols = ['BUSINESS_UNIT_CODE', 'ACCOUNT_CODE', 'BASE_AMOUNT', 'JOURNAL_DATE']
        available = [c for c in dedup_cols if c in df.columns]
        if available:
            df = df.drop_duplicates(subset=available, keep='first')
        
        # Remove rows with all empty text
        available_text = [c for c in text_cols if c in df.columns]
        if available_text:
            mask = df[available_text].notna().any(axis=1)
            df = df[mask]
        
        print(f"Cleaned: {len(df)} rows (removed {initial - len(df)})")
        return df

In [22]:
cleaner = DataCleaner()
raw_data_clean = cleaner.clean(raw_data.copy())
labeled_data_clean = cleaner.clean(labeled_data.copy())

Cleaning 47391 rows...
Cleaned: 33616 rows (removed 13775)
Cleaning 109034 rows...
Cleaned: 33678 rows (removed 75356)


In [9]:
# Check label distribution
print("\nLabel Distribution:")
print("=" * 40)

me_col = 'me_label' if 'me_label' in labeled_data_clean.columns else 'ME_LABEL'
if me_col in labeled_data_clean.columns:
    print(f"\n{me_col}:")
    print(labeled_data_clean[me_col].value_counts(dropna=False))

tax_col = 'tax_description' if 'tax_description' in labeled_data_clean.columns else 'TAX_DESCRIPTION'
if tax_col in labeled_data_clean.columns:
    print(f"\nTop 15 {tax_col}:")
    print(labeled_data_clean[tax_col].value_counts().head(15))


Label Distribution:


NameError: name 'labeled_data_clean' is not defined

## 7. Feature Engineering

In [10]:
class FeatureEngineer:
    """
    Comprehensive feature engineering.
    """
    
    def __init__(self):
        self.location_extractor = LocationExtractor()
    
    def create_combined_text(self, df: pd.DataFrame) -> pd.Series:
        text_cols = ['PURPOSE', 'CHARGE_DESCRIPTION', 'LINE_DESCR',
                     'DESCRIPTION', 'INVOICE_DESCR', 'VENDOR_NAME', 'DESCR']
        available = [c for c in text_cols if c in df.columns]
        
        def combine(row):
            parts = [str(row.get(c, '')).strip() for c in available 
                     if pd.notna(row.get(c)) and str(row.get(c)).strip()]
            return ' '.join(parts)
        
        combined = df.apply(combine, axis=1)
        
        # Clean text
        combined = combined.str.lower()
        combined = combined.str.replace(r'[^a-z0-9\s]', ' ', regex=True)
        combined = combined.str.replace(r'\s+', ' ', regex=True).str.strip()
        
        return combined
    
    def extract_amount_features(self, df: pd.DataFrame) -> pd.DataFrame:
        features = pd.DataFrame(index=df.index)
        
        if 'BASE_AMOUNT' in df.columns:
            amt = df['BASE_AMOUNT'].fillna(0)
            features['amount'] = amt
            features['amount_abs'] = np.abs(amt)
            features['amount_log'] = np.log1p(np.abs(amt))
            features['amount_is_negative'] = (amt < 0).astype(int)
            features['amount_is_round_100'] = (amt % 100 == 0).astype(int)
            features['amount_is_round_50'] = (amt % 50 == 0).astype(int)
            
            # Buckets
            features['amount_bucket'] = pd.cut(
                np.abs(amt),
                bins=[-np.inf, 50, 100, 200, 300, 500, 1000, 5000, np.inf],
                labels=['0-50', '50-100', '100-200', '200-300', '300-500', 
                        '500-1000', '1000-5000', '5000+']
            ).astype(str)
        
        return features
    
    def extract_date_features(self, df: pd.DataFrame) -> pd.DataFrame:
        features = pd.DataFrame(index=df.index)
        
        if 'JOURNAL_DATE' in df.columns:
            dates = pd.to_datetime(df['JOURNAL_DATE'], errors='coerce')
            features['month'] = dates.dt.month.fillna(0).astype(int)
            features['day_of_week'] = dates.dt.dayofweek.fillna(0).astype(int)
            features['day_of_month'] = dates.dt.day.fillna(0).astype(int)
            features['quarter'] = dates.dt.quarter.fillna(0).astype(int)
            features['is_weekend'] = dates.dt.dayofweek.isin([5, 6]).astype(int)
            features['is_month_end'] = (dates.dt.day >= 25).astype(int)
            features['is_friday'] = (dates.dt.dayofweek == 4).astype(int)
            features['is_december'] = (dates.dt.month == 12).astype(int)
            features['is_fbt_q4'] = dates.dt.month.isin([1, 2, 3]).astype(int)
        
        return features
    
    def extract_account_features(self, df: pd.DataFrame) -> pd.DataFrame:
        features = pd.DataFrame(index=df.index)
        
        if 'ACCOUNT_CODE' in df.columns:
            code = df['ACCOUNT_CODE'].fillna(0).astype(int).astype(str)
            features['account_code'] = code
            features['account_group'] = code.str[:4]
            
            # Known entertainment-related accounts
            features['is_entertainment_account'] = df['ACCOUNT_CODE'].isin([5130006000]).astype(int)
            features['is_meals_account'] = df['ACCOUNT_CODE'].isin([5130004000]).astype(int)
            features['is_travel_account'] = df['ACCOUNT_CODE'].isin([5130001000, 5130002000, 5130003000, 5130005000]).astype(int)
        
        return features
    
    def extract_text_features(self, df: pd.DataFrame) -> pd.DataFrame:
        features = pd.DataFrame(index=df.index)
        combined = self.create_combined_text(df)
        
        # Length features
        features['text_length'] = combined.str.len()
        features['word_count'] = combined.str.split().str.len()
        
        # Keyword features
        keyword_groups = {
            'entertainment': ['dinner', 'lunch', 'breakfast', 'meal', 'restaurant',
                              'catering', 'entertainment', 'party', 'celebration',
                              'christmas', 'xmas', 'farewell', 'function', 'drinks',
                              'alcohol', 'wine', 'beer', 'event'],
            'travel': ['flight', 'airfare', 'taxi', 'uber', 'accommodation', 'hotel',
                       'travel', 'airport', 'parking', 'qantas', 'virgin', 'cabcharge'],
            'training': ['training', 'conference', 'seminar', 'workshop', 'course'],
            'recreation': ['team building', 'teambuilding', 'fun day', 'bowling',
                           'golf', 'sailing', 'escape room', 'activity'],
            'client': ['client', 'customer', 'partner', 'stakeholder', 'external'],
            'staff': ['staff', 'employee', 'team', 'internal', 'offsite']
        }
        
        for group, keywords in keyword_groups.items():
            pattern = '|'.join(keywords)
            features[f'has_{group}_kw'] = combined.str.contains(pattern, na=False).astype(int)
        
        return features
    
    def extract_location_features(self, df: pd.DataFrame) -> pd.DataFrame:
        print("Extracting location features...")
        combined = self.create_combined_text(df)
        
        location_features = []
        for i, text in enumerate(combined):
            if i % 5000 == 0:
                print(f"  Processing row {i}/{len(combined)}")
            features = self.location_extractor.extract_location_features(text)
            location_features.append(features)
        
        loc_df = pd.DataFrame(location_features, index=df.index)
        print(f"  Done. Found locations in {(loc_df['locations_found'] > 0).sum()} rows")
        return loc_df
    
    def engineer_all_features(self, df: pd.DataFrame) -> Tuple[pd.Series, pd.DataFrame]:
        print("\nEngineering features...")
        
        combined_text = self.create_combined_text(df)
        print(f"  Combined text created")
        
        amount_feat = self.extract_amount_features(df)
        print(f"  Amount features: {amount_feat.shape[1]}")
        
        date_feat = self.extract_date_features(df)
        print(f"  Date features: {date_feat.shape[1]}")
        
        account_feat = self.extract_account_features(df)
        print(f"  Account features: {account_feat.shape[1]}")
        
        text_feat = self.extract_text_features(df)
        print(f"  Text features: {text_feat.shape[1]}")
        
        location_feat = self.extract_location_features(df)
        print(f"  Location features: {location_feat.shape[1]}")
        
        # Combine all features
        all_features = pd.concat([
            amount_feat, date_feat, account_feat, text_feat, location_feat
        ], axis=1)
        
        print(f"\nTotal features: {all_features.shape[1]}")
        return combined_text, all_features

In [11]:
# Engineer features
feature_engineer = FeatureEngineer()
labeled_text, labeled_features = feature_engineer.engineer_all_features(labeled_data_clean)
labeled_data_clean['combined_text'] = labeled_text

NameError: name 'labeled_data_clean' is not defined

In [ ]:
# Preview features
print("\nFeature Preview:")
print(labeled_features.head())

In [ ]:
# Location statistics
print("\nLocation Statistics:")
print(f"Rows with locations: {(labeled_features['locations_found'] > 0).sum()}")
print(f"\nTravel Category Distribution:")
print(labeled_features['travel_category'].value_counts())
print(f"\nDistance Statistics (km):")
print(labeled_features['distance_from_ref_km'].describe())

## 8. Prepare Training Data

In [ ]:
def prepare_training_data(df: pd.DataFrame, features: pd.DataFrame, 
                          text_col: str = 'combined_text'):
    """
    Prepare data for supervised training.
    """
    # Find label column
    label_col = 'me_label' if 'me_label' in df.columns else 'ME_LABEL'
    
    # Filter valid labels
    valid_mask = df[label_col].isin(['Y', 'N'])
    df_valid = df[valid_mask].copy()
    features_valid = features[valid_mask].copy()
    
    print(f"Valid samples: {len(df_valid)}")
    print(f"Label distribution:\n{df_valid[label_col].value_counts()}")
    
    # Prepare data
    X_text = df_valid[text_col].values
    X_features = features_valid.copy()
    y = df_valid[label_col].values
    
    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    print(f"Classes: {label_encoder.classes_}")
    
    return X_text, X_features, y_encoded, label_encoder, df_valid

In [ ]:
X_text, X_features, y, label_encoder, train_df = prepare_training_data(
    labeled_data_clean, labeled_features
)

In [ ]:
# Split data
X_text_train, X_text_test, X_feat_train, X_feat_test, y_train, y_test = train_test_split(
    X_text, X_features, y,
    test_size=CONFIG['test_size'],
    random_state=CONFIG['random_state'],
    stratify=y
)

print(f"Training: {len(X_text_train)}")
print(f"Test: {len(X_text_test)}")

## 9. Text Vectorization

In [ ]:
# TF-IDF Vectorizer
tfidf = TfidfVectorizer(
    max_features=CONFIG['max_features'],
    ngram_range=CONFIG['ngram_range'],
    min_df=CONFIG['min_df'],
    max_df=CONFIG['max_df'],
    sublinear_tf=True,
    strip_accents='unicode',
    lowercase=True
)

X_tfidf_train = tfidf.fit_transform(X_text_train)
X_tfidf_test = tfidf.transform(X_text_test)

print(f"TF-IDF shape: {X_tfidf_train.shape}")

In [ ]:
# Prepare numerical features
numerical_cols = X_feat_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = ['amount_bucket', 'travel_category', 'account_code', 'account_group']
categorical_cols = [c for c in categorical_cols if c in X_feat_train.columns]

print(f"Numerical features: {len(numerical_cols)}")
print(f"Categorical features: {len(categorical_cols)}")

In [ ]:
# Scale numerical features
scaler = StandardScaler()

X_num_train = X_feat_train[numerical_cols].fillna(0).values
X_num_test = X_feat_test[numerical_cols].fillna(0).values

X_num_train_scaled = scaler.fit_transform(X_num_train)
X_num_test_scaled = scaler.transform(X_num_test)

# Encode categorical features
if categorical_cols:
    cat_encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
    X_cat_train = cat_encoder.fit_transform(X_feat_train[categorical_cols].fillna('unknown'))
    X_cat_test = cat_encoder.transform(X_feat_test[categorical_cols].fillna('unknown'))
else:
    X_cat_train = None
    X_cat_test = None

In [ ]:
# Combine all features
from scipy import sparse

feature_matrices_train = [X_tfidf_train, sparse.csr_matrix(X_num_train_scaled)]
feature_matrices_test = [X_tfidf_test, sparse.csr_matrix(X_num_test_scaled)]

if X_cat_train is not None:
    feature_matrices_train.append(X_cat_train)
    feature_matrices_test.append(X_cat_test)

X_combined_train = sparse.hstack(feature_matrices_train)
X_combined_test = sparse.hstack(feature_matrices_test)

print(f"Combined feature matrix: {X_combined_train.shape}")

## 10. Model Training

In [ ]:
# Define models
models = {
    'Logistic Regression': LogisticRegression(
        max_iter=1000, random_state=42, class_weight='balanced', n_jobs=-1
    ),
    'Complement NB': ComplementNB(alpha=0.1),
    'Linear SVC': CalibratedClassifierCV(
        LinearSVC(max_iter=2000, random_state=42, class_weight='balanced')
    ),
    'SGD Classifier': SGDClassifier(
        loss='modified_huber', max_iter=1000, random_state=42, 
        class_weight='balanced', n_jobs=-1
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=200, max_depth=20, random_state=42,
        class_weight='balanced', n_jobs=-1
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=100, max_depth=5, random_state=42
    ),
    'Extra Trees': ExtraTreesClassifier(
        n_estimators=200, max_depth=20, random_state=42,
        class_weight='balanced', n_jobs=-1
    )
}

In [ ]:
# Train and evaluate models
results = []

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Cross-validation
    cv = StratifiedKFold(n_splits=CONFIG['cv_folds'], shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_combined_train, y_train, cv=cv, scoring='f1')
    
    # Train
    model.fit(X_combined_train, y_train)
    
    # Predict
    y_pred = model.predict(X_combined_test)
    
    # Metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    results.append({
        'Model': name,
        'CV F1 Mean': cv_scores.mean(),
        'CV F1 Std': cv_scores.std(),
        'Test Accuracy': accuracy,
        'Test F1': f1
    })
    
    print(f"  CV F1: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")
    print(f"  Test F1: {f1:.4f}")

In [ ]:
# Results comparison
results_df = pd.DataFrame(results).sort_values('Test F1', ascending=False)
print("\nModel Comparison:")
print(results_df.to_string(index=False))

In [ ]:
# Visualize results
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(results_df))
width = 0.35

ax.bar(x - width/2, results_df['CV F1 Mean'], width, label='CV F1', alpha=0.8)
ax.bar(x + width/2, results_df['Test F1'], width, label='Test F1', alpha=0.8)

ax.set_ylabel('F1 Score')
ax.set_title('Model Comparison')
ax.set_xticks(x)
ax.set_xticklabels(results_df['Model'], rotation=45, ha='right')
ax.legend()
ax.set_ylim(0, 1)
plt.tight_layout()
plt.show()

## 11. Best Model Analysis

In [ ]:
# Select best model
best_name = results_df.iloc[0]['Model']
best_model = models[best_name]

print(f"Best Model: {best_name}")

y_pred = best_model.predict(X_combined_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

In [ ]:
# Confusion Matrix
fig, ax = plt.subplots(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
disp.plot(ax=ax, cmap='Blues', values_format='d')
ax.set_title(f'Confusion Matrix - {best_name}')
plt.tight_layout()
plt.show()

## 12. Unsupervised Clustering (Optional)

In [ ]:
if CONFIG.get('enable_clustering', False):
    print("Running clustering analysis...")
    
    # Reduce dimensionality for clustering
    svd = TruncatedSVD(n_components=100, random_state=42)
    X_reduced = svd.fit_transform(X_combined_train)
    
    # K-Means clustering
    n_clusters = CONFIG['n_clusters']
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_reduced)
    
    # Evaluate clustering
    silhouette = silhouette_score(X_reduced, cluster_labels)
    calinski = calinski_harabasz_score(X_reduced, cluster_labels)
    
    print(f"\nClustering Metrics:")
    print(f"  Silhouette Score: {silhouette:.4f}")
    print(f"  Calinski-Harabasz Score: {calinski:.2f}")
    
    # Cluster distribution
    print(f"\nCluster Distribution:")
    print(pd.Series(cluster_labels).value_counts().sort_index())
    
    # Visualize with t-SNE
    print("\nGenerating t-SNE visualization...")
    tsne = TSNE(n_components=2, random_state=42, perplexity=30)
    X_tsne = tsne.fit_transform(X_reduced[:2000])  # Limit for speed
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Clusters
    scatter = axes[0].scatter(X_tsne[:, 0], X_tsne[:, 1], c=cluster_labels[:2000], 
                              cmap='tab10', alpha=0.6, s=10)
    axes[0].set_title('t-SNE: Cluster Labels')
    plt.colorbar(scatter, ax=axes[0])
    
    # True labels
    scatter = axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y_train[:2000],
                              cmap='RdYlGn', alpha=0.6, s=10)
    axes[1].set_title('t-SNE: True Labels (Y=Red, N=Green)')
    plt.colorbar(scatter, ax=axes[1])
    
    plt.tight_layout()
    plt.show()
else:
    print("Clustering disabled. Set CONFIG['enable_clustering'] = True to enable.")

## 13. Feature Importance

In [ ]:
def get_top_features(model, tfidf_vectorizer, n=30):
    """Get top features from model."""
    feature_names = list(tfidf_vectorizer.get_feature_names_out())
    feature_names.extend(numerical_cols)
    
    if hasattr(model, 'coef_'):
        coef = model.coef_[0] if len(model.coef_.shape) > 1 else model.coef_
        coef = coef[:len(feature_names)]  # Limit to named features
        
        # Top positive (ME = Y)
        top_pos_idx = np.argsort(coef)[-n:]
        top_pos = [(feature_names[i], coef[i]) for i in reversed(top_pos_idx) if i < len(feature_names)]
        
        # Top negative (ME = N)
        top_neg_idx = np.argsort(coef)[:n]
        top_neg = [(feature_names[i], coef[i]) for i in top_neg_idx if i < len(feature_names)]
        
        return top_pos, top_neg
    
    elif hasattr(model, 'feature_importances_'):
        importance = model.feature_importances_[:len(feature_names)]
        top_idx = np.argsort(importance)[-n:]
        top = [(feature_names[i], importance[i]) for i in reversed(top_idx) if i < len(feature_names)]
        return top, []
    
    return [], []

In [ ]:
# Get feature importance for best model
if hasattr(best_model, 'coef_') or hasattr(best_model, 'feature_importances_'):
    top_pos, top_neg = get_top_features(best_model, tfidf)
    
    print("Top Features for ME = Y (FBT Subject):")
    print("-" * 50)
    for feat, score in top_pos[:20]:
        print(f"  {feat:35s} {score:+.4f}")
    
    if top_neg:
        print("\nTop Features for ME = N (Not FBT Subject):")
        print("-" * 50)
        for feat, score in top_neg[:20]:
            print(f"  {feat:35s} {score:+.4f}")
else:
    print("Feature importance not available for this model type.")

## 14. Hyperparameter Tuning

In [ ]:
# Grid search for Logistic Regression
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['saga']
}

grid_search = GridSearchCV(
    LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced'),
    param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

print("Running GridSearchCV...")
grid_search.fit(X_combined_train, y_train)

print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best CV F1: {grid_search.best_score_:.4f}")

In [ ]:
# Evaluate tuned model
tuned_model = grid_search.best_estimator_
y_pred_tuned = tuned_model.predict(X_combined_test)

print("\nTuned Model Results:")
print(classification_report(y_test, y_pred_tuned, target_names=label_encoder.classes_))

## 15. Ensemble Model

In [ ]:
# Create ensemble
ensemble = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')),
        ('nb', ComplementNB(alpha=0.1)),
        ('svc', CalibratedClassifierCV(LinearSVC(max_iter=2000, random_state=42, class_weight='balanced'))),
        ('rf', RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1))
    ],
    voting='soft'
)

print("Training ensemble...")
ensemble.fit(X_combined_train, y_train)

y_pred_ensemble = ensemble.predict(X_combined_test)
print("\nEnsemble Results:")
print(classification_report(y_test, y_pred_ensemble, target_names=label_encoder.classes_))

## 16. Save Model

In [ ]:
# Save all components
model_package = {
    'model': ensemble,
    'tfidf': tfidf,
    'scaler': scaler,
    'cat_encoder': cat_encoder if categorical_cols else None,
    'label_encoder': label_encoder,
    'numerical_cols': numerical_cols,
    'categorical_cols': categorical_cols,
    'config': CONFIG
}

joblib.dump(model_package, CONFIG['model_output'])
print(f"Model saved to: {CONFIG['model_output']}")

## 17. Inference Function

In [ ]:
def predict_fbt(texts: List[str], features_df: pd.DataFrame = None,
                model_path: str = CONFIG['model_output']) -> pd.DataFrame:
    """
    Predict FBT classification for new expenses.
    """
    # Load model
    pkg = joblib.load(model_path)
    
    # Transform text
    X_tfidf = pkg['tfidf'].transform(texts)
    
    # Handle features
    if features_df is not None:
        X_num = features_df[pkg['numerical_cols']].fillna(0).values
        X_num_scaled = pkg['scaler'].transform(X_num)
        
        if pkg['cat_encoder']:
            X_cat = pkg['cat_encoder'].transform(features_df[pkg['categorical_cols']].fillna('unknown'))
            X_combined = sparse.hstack([X_tfidf, sparse.csr_matrix(X_num_scaled), X_cat])
        else:
            X_combined = sparse.hstack([X_tfidf, sparse.csr_matrix(X_num_scaled)])
    else:
        X_combined = X_tfidf
    
    # Predict
    predictions = pkg['model'].predict(X_combined)
    labels = pkg['label_encoder'].inverse_transform(predictions)
    
    # Probabilities
    if hasattr(pkg['model'], 'predict_proba'):
        probs = pkg['model'].predict_proba(X_combined)
        prob_y = probs[:, 1]  # Probability of Y (ME)
    else:
        prob_y = np.nan
    
    return pd.DataFrame({
        'text': texts,
        'prediction': labels,
        'prob_ME': prob_y
    })

In [ ]:
# Test inference
test_texts = [
    "client dinner at restaurant melbourne 5 partners",
    "taxi to airport business trip brisbane",
    "team building activity bowling sydney",
    "training seminar registration fee",
    "christmas party catering staff 50 people",
    "flight to roma client visit farm",
    "lunch meeting with client local cafe"
]

predictions = predict_fbt(test_texts)
print("\nSample Predictions:")
print(predictions.to_string(index=False))

## 18. Summary

In [ ]:
print("=" * 60)
print("PIPELINE SUMMARY")
print("=" * 60)
print(f"\nData:")
print(f"  Raw samples: {len(raw_data_clean)}")
print(f"  Labeled samples: {len(train_df)}")
print(f"  Training samples: {len(X_text_train)}")
print(f"  Test samples: {len(X_text_test)}")
print(f"\nFeatures:")
print(f"  TF-IDF features: {X_tfidf_train.shape[1]}")
print(f"  Numerical features: {len(numerical_cols)}")
print(f"  Categorical features: {len(categorical_cols)}")
print(f"  Total features: {X_combined_train.shape[1]}")
print(f"\nBest Model: {best_name}")
print(f"  Test F1: {results_df.iloc[0]['Test F1']:.4f}")
print(f"\nLocation Features:")
print(f"  Locations database: {len(AUSTRALIAN_LOCATIONS)} entries")
print(f"  Samples with locations: {(labeled_features['locations_found'] > 0).sum()}")
print(f"\nSaved:")
print(f"  Model: {CONFIG['model_output']}")